In [324]:
import pandas as pd
import json
import math

In [325]:
data = pd.read_csv("./assets/data.csv", header=None, names=['date','item', 'debit', 'credit','subCategory'],index_col=False)
maps = pd.read_csv("./rules/1to1maps.csv", header=None, names=['item', 'subCategory'])
subCategories = pd.read_csv("./rules/categories.csv", header=None, names=['item', 'subCategory'])
categories = pd.read_csv("./rules/breakdown.csv", header=None, names=['subCategory', 'category'])

categoryMap ={}
for i, row in categories.iterrows():
    categoryMap[row['subCategory']] = row['category']



In [326]:
def check1to1(x):
    try:
        index = pd.Index(maps['item']).get_loc(x.rstrip())   
        return maps.loc[index]['subCategory']
    except:
        return None

data['subCategory'] = data['subCategory'].combine_first(data['item'].apply(check1to1))

In [335]:
def check(x):
    for i, categoryRow in subCategories.iloc[::-1].iterrows():
        if categoryRow['item'] in x:
            return categoryRow['subCategory']
    return None

data['test'] = data['item'].apply(check)


In [340]:
data[data['test']==""]

,date,item,debit,credit,subCategory,balance,test


In [333]:
# then mapping general categories
def tryc1():
    for i, categoryRow in subCategories.iloc[::-1].iterrows():
        indo = ((data['item'].str.contains(categoryRow['item'])) & (data['subCategory']==""))
        subCatArray[indo] = categoryRow['subCategory']

    
def tryc2():
    def check(x):
        for i, categoryRow in subCategories.iloc[::-1].iterrows():
            if categoryRow['item'] in x:
                return categoryRow['subCategory']
        return None
        
    data['test'] = data['item'].apply(check)


In [334]:
%timeit tryc1()

KeyboardInterrupt: 

In [47]:
data['balance']=data['credit']-data['debit']

#finally taking care of special categories with logic
specialCategories = subCategories[subCategories['item'].str.contains('{{')]
for i, categoryRow in specialCategories.iterrows():
    itemValuePair = categoryRow['item'].replace('}}', '').split('{{')
    indo = (data['item'].str.contains(itemValuePair[0].rstrip()) & (data['balance']==(float(itemValuePair[1]))))
    subCatArray[indo] = categoryRow['subCategory']

data['subCategory'] = subCatArray
data['category'] = data['subCategory'].map(categoryMap)
data['year']= pd.to_datetime(data['date']).dt.year
data['month']= pd.to_datetime(data['date']).dt.month


In [16]:
dataWithCategory = (data[data['subCategory'] != ""])
dataWithoutCategory = (data[data['subCategory'] == ""])
dataWithCategory[['item','category','subCategory','date','year','month','debit','credit','balance']].sort_values(by='date', ascending=False).to_csv('./processed/processed.csv', index=False)
dataWithoutCategory[['item','date','balance']].to_csv('./processed/not_found.csv')

print('unCategorized items')
print(dataWithoutCategory[['item','date','balance']])


unCategorized items
                                                   item        date  balance
3081     ONE CUT VINYL 8778877815 217.30 GBP @ 2.068890  2016-02-01  -449.57
3420    HANANOMAI KAYABACHO TOKYO 4811.00 JPY @ .010295  2015-06-25   -49.53
3422  LIMOUSINE BUS NARITA2 KIT CHIBA 3800.00 JPY @ ...  2015-06-24   -39.11
3701  WILDFIRE (THE PEAK) (1203 THE PEAK 162.80 HKD ...  2014-12-12   -24.86
4894  Point of Sale - Interac RETAIL PURCHASE 000001...  2018-01-11   -11.00
4901  Point of Sale - Interac RETAIL PURCHASE 000068...  2018-01-08   -28.20
4912  AIR FRANCE 0571409230279 ROISSY CDG CE 87.67 E...  2018-01-26  -138.51
4913  AIR FRANCE 0571409230280 ROISSY CDG CE 87.67 E...  2018-01-26  -138.51
4914  AIR FRANCE 0578252207759 ROISSY CDG CE 12.00 E...  2018-01-26   -18.96
4916  EASYJET 000 ETJ5X8 LUTON, BEDS 90.28 EUR @ 1.5...  2018-01-25  -141.88
4930                 SHEHERZADE PERSIAN GRI TORONTO, ON  2018-01-12   -22.32
4932                       WOW AIR EHF 012NVM REYKJAVIK 

In [230]:
# TESTING TWO FILES
old = pd.read_csv("./processed/output-old.csv", index_col=False)
new = pd.read_csv("./processed/processed.csv", index_col=False)
old.sort_values(by=['date','debit'], inplace=True)
new.sort_values(by=['date','debit'], inplace=True)
old.reset_index( drop=True,inplace=True)
new.reset_index(drop=True,inplace=True)
ineq = []
for i, row in old.iterrows():
    if (row['subCategory'] != new.loc[i]['subCategory']):
        if(row['item']=="TIGERDIRECT.CA MARKHAM, ON"):
            pass
        elif('WINNERSHOMESENSE' in row['item']):
            pass
        else:
            pass
#             print(f"{row['subCategory']}, {new.loc[i]['subCategory']} - {row['balance']} {row['item']}{new.loc[i]['item']}")
            

y = new.groupby('subCategory').sum()['balance']
x = old.groupby('subCategory').sum()['balance']
for i,value in enumerate(x):
    if(value != y[i]):
        print(value, y[i])

29935.72 29935.72
-5958.06 -5856.37
28060.0 29630.0
-8394.98 -8418.56
-11333.58 -11435.27
-18.95 -1588.95


In [318]:
# one to one map benchmarking
def try1():    
    def check1to1(x):
        try:
            index = pd.Index(maps['item']).get_loc(x.rstrip())   
            return maps.loc[index]['subCategory']
        except:
            return None

    data['subCategory'] = data['subCategory'].combine_first(data['item'].apply(check1to1))
    
def try2():
    data.fillna("", inplace=True)
    subCatArray = []
    # first mapping the 1to1 mappings
    for i, row in data.iterrows():
        if (row['subCategory'] != ""):
            subCatArray.append(row['subCategory'])
        else:
            try:
                index = pd.Index(maps['item']).get_loc(row['item'].rstrip())
                subCategory = maps.loc[index]['subCategory']
                subCatArray.append(subCategory)
            except:
                subCatArray.append("")
    data['subCategory'] = subCatArray


In [319]:
%timeit try1()

483 ms ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [321]:
%timeit try2()

696 ms ± 2.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
